<a href="https://colab.research.google.com/github/estarriol1999/AI-cup/blob/master/SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/SC')
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
MIR-ST500  ST_1.pt  ST_3.pt  ST_5.pt  ST_7.pt  train_11.pkl
ST_0.pt    ST_2.pt  ST_4.pt  ST_6.pt  ST_8.pt  train_all.pkl


In [0]:
import json
import os
import numpy as np
import pickle
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.utils.rnn as rnn_utils

class Myrnn(nn.Module):
    def __init__(self, input_dim, hidden_size= 100):
        super(Myrnn, self).__init__()
        self.hidden_size = hidden_size

        self.Linear1 = nn.Linear(input_dim, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers= 5, bidirectional= True)
        self.Linear2 = nn.Linear(hidden_size* 2, 2)
        self.Linear3 = nn.Linear(hidden_size* 2, 1)

    def forward(self, input_data):
        out = F.relu(self.Linear1(input_data))
        out, hidden = self.rnn(out)
        #out1 is for onset & offset
        out1 = torch.sigmoid(self.Linear2(out))
        #out2 is for pitch
        out2 = self.Linear3(out)
        return out1, out2

class MyData(Data.Dataset):
    def __init__(self, data_seq, label):
        self.data_seq = data_seq
        self.label= label

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return {
            'data': self.data_seq[idx],
            'label': self.label[idx]
        }

def collate_fn(samples):
    batch = {}
    #print (samples[0]['data'].shape)
    temp= [torch.from_numpy(np.array(sample['data'], dtype= np.float32)) for sample in samples]
    padded_data = rnn_utils.pad_sequence(temp, batch_first=True, padding_value= 0)
    batch['data']= padded_data
    batch['label']= [np.array(sample['label'], dtype= np.float32) for sample in samples]

    return batch

def post_processing(output1, pitch):
    pitch= pitch.squeeze(1).squeeze(1).cpu().detach().numpy()
    print (pitch.shape)
    print (torch.mean(output1))
    threshold= 0.1
    notes= []
    this_onset= None
    this_offset= None
    this_pitch= None

    for i in range(len(output1)):
        if output1[i][0][0] > threshold and this_onset == None:
            this_onset= i
        elif output1[i][0][1] > threshold and this_onset != None and this_onset+ 1 < i and this_offset == None:
            this_offset= i
            this_pitch= int(round(np.mean(pitch[this_onset:this_offset+ 1])))
            notes.append([this_onset* 0.032+ 0.016, this_offset* 0.032+ 0.016, this_pitch])
            this_onset= None
            this_offset= None
            this_pitch= None

    print (np.array(notes))
    return notes

def testing(net, sample, device):
    net.eval()
    data = sample['data']
    data= torch.Tensor(data)

    target= sample['label']
    target= torch.Tensor(target)

    data= data.unsqueeze(1)
    target= target.unsqueeze(1)

    print (data.shape)
    print (target.shape)

    data_length= list(data.shape)[0]

    data = data.to(device, dtype=torch.float)
    target = target.to(device, dtype=torch.float)

    output1, output2 = net(data)
    print (output1.shape)
    print (output2.shape)
    answer = post_processing(output1, output2)
    return answer

def do_training(net, loader, optimizer, device):

    num_epoch = 50
    criterion_onset= nn.BCELoss()
    criterion_pitch= nn.L1Loss()
    train_loss= 0.0
    total_length= 0

    for epoch in range(num_epoch):
        net.train()
        total_length= 0.0
        print ("epoch %d start time: %f" %(epoch, time.time()))
        train_loss= 0.0

        for batch_idx, sample in enumerate(loader):
            data = sample['data']
            data= torch.Tensor(data)

            target= sample['label']
            target= torch.Tensor(target)

            data= data.permute(1,0,2)
            target= target.permute(1,0,2)

            #print (data.shape)
            #print (target.shape)
            data_length= list(data.shape)[0]

            data = data.to(device, dtype=torch.float)
            target = target.to(device, dtype=torch.float)

            optimizer.zero_grad()
            output1, output2 = net(data)
            #print (output1)
            #print (output2)

            #print (output1.shape)
            #print (output2.shape)

            total_loss= criterion_onset(output1, torch.narrow(target, dim= 2, start= 0, length= 2))
            total_loss= total_loss+ criterion_pitch(output2, torch.narrow(target, dim= 2, start= 2, length= 1))
            train_loss= train_loss+ total_loss.item()
            total_length= total_length+ 1
            total_loss.backward()
            optimizer.step()

            if batch_idx % 50 == 0:
                print ("epoch %d, sample %d, loss %.6f" %(epoch, batch_idx, total_loss))
                #print ("current time: %f" %(time.time()))
                sys.stdin.flush()
        print('epoch %d, avg loss: %.6f' %(epoch, train_loss/ total_length))
        model_path= f'ori_{epoch}.pt'
        torch.save(net.state_dict(), model_path)
    return net

def preprocess(data_seq, label):
    new_label= []
    for i in range(len(label)):
        label_of_one_song= []
        cur_note= 0
        cur_note_onset= label[i][cur_note][0]
        cur_note_offset= label[i][cur_note][1]
        cur_note_pitch= label[i][cur_note][2]

        for j in range(len(data_seq[i])):
            cur_time= j* 0.032+ 0.016
        
            if abs(cur_time - cur_note_onset) < 0.017:
                label_of_one_song.append(np.array([1, 0, cur_note_pitch]))

            elif cur_time < cur_note_onset or cur_note >= len(label[i]):
                label_of_one_song.append(np.array([0, 0, 0.0]))

            elif abs(cur_time - cur_note_offset) < 0.017:
                label_of_one_song.append(np.array([0, 1, cur_note_pitch]))
                cur_note= cur_note+ 1
                if cur_note < len(label[i]):
                    cur_note_onset= label[i][cur_note][0]
                    cur_note_offset= label[i][cur_note][1]
                    cur_note_pitch= label[i][cur_note][2]
            else:
                label_of_one_song.append(np.array([0, 0, cur_note_pitch]))

        new_label.append(label_of_one_song)

    return new_label

if __name__ == '__main__':
    '''
    THE_FOLDER = "./MIR-ST500"
    data_seq= []
    label= []

    for the_dir in os.listdir(THE_FOLDER):
        #print (the_dir)
        if not os.path.isdir(THE_FOLDER + "/" + the_dir):
            continue

        json_path = THE_FOLDER + "/" + the_dir+ f"/{the_dir}_feature.json"
        gt_path= THE_FOLDER+ "/" +the_dir+ "/"+ the_dir+ "_groundtruth.txt"
        youtube_link_path= THE_FOLDER+ "/" + the_dir+ "/"+ the_dir+ "_link.txt"

        with open(json_path, 'r') as json_file:
            temp = json.loads(json_file.read())

        gtdata = np.loadtxt(gt_path)

        data= []
        for key, value in temp.items():
            data.append(value)

        data= np.array(data).T

        data_seq.append(data)
        label.append(gtdata)
    
    label= preprocess(data_seq, label)
    train_data = MyData(data_seq, label)
    
    with open("train_all_2.pkl", 'wb') as pkl_file:
        pickle.dump(train_data, pkl_file)
    '''
    with open("train_all_2.pkl", 'rb') as pkl_file:
        train_data= pickle.load(pkl_file)

    input_dim= 23
    hidden_size= 50
    BATCH_SIZE= 1
    loader = Data.DataLoader(dataset=train_data, batch_size= BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    model = Myrnn(input_dim, hidden_size)
    optimizer = optim.Adam(model.parameters(), lr= 0.001)

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    else: 
        device = 'cpu'
    print("use",device,"now!")

    model.to(device)
    model= do_training(model, loader, optimizer, device)
    #for testing
    
    #model.load_state_dict(torch.load("ST_5.pt"))
    #testing(model, train_data[0], device)

use cuda now!
epoch 0 start time: 1591606475.887910
epoch 0, sample 0, loss 33.396461
epoch 0, sample 50, loss 39.438618
epoch 0, sample 100, loss 31.910355
epoch 0, sample 150, loss 21.469341
epoch 0, sample 200, loss 20.027613
epoch 0, sample 250, loss 19.050505
epoch 0, sample 300, loss 27.896547
epoch 0, sample 350, loss 27.752306
epoch 0, sample 400, loss 19.586676
epoch 0, sample 450, loss 10.502892
epoch 0, avg loss: 23.511831
epoch 1 start time: 1591609315.619879
epoch 1, sample 0, loss 9.961915
epoch 1, sample 50, loss 7.468236
epoch 1, sample 100, loss 10.143873
epoch 1, sample 150, loss 7.004801
epoch 1, sample 200, loss 7.569450
epoch 1, sample 250, loss 6.306088
epoch 1, sample 300, loss 6.442085
epoch 1, sample 350, loss 4.957866
epoch 1, sample 400, loss 7.222989
epoch 1, sample 450, loss 9.548290
epoch 1, avg loss: 8.724917
epoch 2 start time: 1591612149.654126
epoch 2, sample 0, loss 6.955619
epoch 2, sample 50, loss 12.960403
epoch 2, sample 100, loss 7.016714
epoch 2